In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch
# 下载数据集的路径
path = './DataSet'
# 使用的设备
device = torch.device("cuda")
# 数据预处理,转换为张量并归一化
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 将图像大小调整为 224x224
    transforms.ToTensor(),  # 将图像转换为张量
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 归一化处理

train_set = CIFAR10(root=path, train=True, download=True, transform=transform)
test_set = CIFAR10(root=path, train=False, download=True, transform=transform)
# 加载数据集并设置批次大小
train_loader = DataLoader(train_set, batch_size=256, shuffle=True)
test_loader = DataLoader(test_set, batch_size=256, shuffle=False)
# 打印数据集的信息
print(
    f"训练集的 batch 数量: {len(train_loader)}，每个 batch 中的数据数量: {train_loader.batch_size}")
print(
    f"测试集的 batch 数量: {len(test_loader)}，每个 batch 中的数据数量: {test_loader.batch_size}")

In [ ]:
import torch
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import Flatten
from torch.nn import Sequential
from torch.nn import BatchNorm2d
from torch.nn import AdaptiveAvgPool2d
from torch.nn import AvgPool2d
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_


class DenseNet(Module):
    class DenseBlock(Module):
        def DenseNet_block(self, input_channels, num_channels):
            return Sequential(
                BatchNorm2d(input_channels),
                ReLU(),
                Conv2d(input_channels, num_channels, kernel_size=3, padding=1)
            )
        def transition_block(self, input_channels, num_channels):
            return Sequential(
                BatchNorm2d(input_channels),
                ReLU(),
                Conv2d(input_channels, num_channels, kernel_size=1),
                AvgPool2d(kernel_size=2, stride=2)
            )

        def __init__(self, num_convs, input_channels, num_channels):
            super().__init__()
            layer = []
            for i in range(num_convs):
                layer.append(self.DenseNet_block(num_channels * i + input_channels, num_channels))
            self.net = Sequential(*layer)

        def forward(self, X):
            for blk in self.net:
                Y = blk(X)
                X = torch.cat((X, Y), dim=1)
            return X

    def __init__(self, num_classes, num_dense_blocks=4, num_convs_per_block=4, growth_rate=32):
        super().__init__()
        self.features = Sequential(
            Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            BatchNorm2d(64),
            ReLU(),
            MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        num_channels = 64
        for i in range(num_dense_blocks):
            dense_block = self.DenseBlock(num_convs_per_block, num_channels, growth_rate)
            self.features.add_module(f'dense_block_{i}', dense_block)
            num_channels += num_convs_per_block * growth_rate
            if i != num_dense_blocks - 1:
                # 通过实例调用 transition_block 方法
                transition = dense_block.transition_block(num_channels, num_channels // 2)
                self.features.add_module(f'transition_block_{i}', transition)
                # 将 num_channels 的值更新为原来的一半
                num_channels = num_channels // 2
        self.features.add_module('global_avg_pool', AdaptiveAvgPool2d((1, 1)))
        self.classifier = Sequential(
            Flatten(),
            Linear(num_channels, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
# 训练模型
# 定义损失函数和优化器
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
import time


def train_model(train_loader, model):
    # 定义损失函数和优化器
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # 训练的轮数
    num_epochs = 10
    # 初始化 running_loss
    running_loss = 0.0
    # 记录训练开始时间
    start_time = time.time()
    # 遍历epoch
    for epoch in range(num_epochs):
        # 遍历训练数据
        for i, (inputs, labels) in enumerate(train_loader):
            # 将数据移动到GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            # 梯度清零
            optimizer.zero_grad()
            # 计算模型输出
            y_hat = model(inputs)
            # 计算损失
            loss = criterion(y_hat, labels)
            # 反向传播
            loss.backward()
            # 更新参数
            optimizer.step()
            running_loss += loss.item()
        # 打印每个 epoch 的损失
        print(
            f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')
        # 每个 epoch 结束后重置 running_loss
        running_loss = 0.0
    # 记录训练结束时间
    end_time = time.time()
    # 计算训练总时间
    total_time = end_time - start_time
    # 转换为分钟和秒
    minutes = int(total_time // 60)
    seconds = total_time % 60
    print(
        f'Training completed in {minutes} minutes and {seconds:.2f} seconds.')

In [ ]:
# 评估模型
from sklearn.metrics import accuracy_score
from numpy import vstack
from numpy import argmax


def evaluate_model(test_loader, model):
    predictions, actuals = list(), list()
    for i, (inputs, labels) in enumerate(test_loader):
        # 将数据移动到GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 计算模型输出
        y_hat = model(inputs)
        # 转换为 numpy 数据类型
        y_hat = y_hat.detach().cpu().numpy()
        actual = labels.cpu().numpy()
        # 转换为类标签
        y_hat = argmax(y_hat, axis=1)
        # 为stack格式化
        actual = actual.reshape((len(actual), 1))
        y_hat = y_hat.reshape((len(y_hat), 1))
        # 存储
        predictions.append(y_hat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # 计算准确率
    acc = accuracy_score(actuals, predictions)
    return acc

In [ ]:
# 产生实例,并且将实例放入GPU
#DenseNet_model = DenseNet(num_classes=10).to(device)
DenseNet_model2 = DenseNet(num_classes=10, num_dense_blocks=5, num_convs_per_block=5, growth_rate=48).to(device)
# 训练实例
train_model(train_loader, DenseNet_model2)
# 评估实例整体准确率
acc = evaluate_model(test_loader, DenseNet_model2)
print('Overall Accuracy: %.3f' % acc)

In [ ]:
# 保存训练好的模型
PATH = './DataSet/model/DenseNet_model2.pth'
torch.save(DenseNet_model2.state_dict(), PATH)

In [ ]:
DenseNet_model2 = DenseNet(num_classes=10, num_dense_blocks=5, num_convs_per_block=5, growth_rate=48).to(device)
DenseNet_model2.load_state_dict(torch.load(PATH, weights_only=True))
classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')
# 初始化分类准确率统计
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 评估每个类别的准确率
with torch.no_grad():
    DenseNet_model2.eval()  # 确保模型处于评估模式
    for data in test_loader:
        images, labels = data
        # 将数据移动到GPU
        images, labels = images.to(device), labels.to(device)
        outputs = DenseNet_model2(images)  # 使用实例化模型
        _, predictions = torch.max(outputs, 1)

        # 将预测结果和标签移动到CPU并转换为numpy数组
        labels = labels.cpu().numpy()
        predictions = predictions.cpu().numpy()

        for label_idx, pred_idx in zip(labels, predictions):
            class_name = classes[label_idx]
            if label_idx == pred_idx:
                correct_pred[class_name] += 1
            total_pred[class_name] += 1

# 打印每个类别的准确率
for classname in classes:
    total = total_pred[classname]
    correct = correct_pred[classname]
    if total == 0:
        print(f"Class {classname} has no test samples")
        continue
    accuracy = 100 * correct / total
    print(f'Accuracy for {classname:5s}: {accuracy:.1f}%')